In [8]:
import pandas as pd
import pickle
from tqdm.auto import tqdm
import time

%load_ext autoreload
%autoreload 2
from question_to_rank import Entity, QuestionToRank
from kbqa.logger import get_logger

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
logger = get_logger()

In [2]:
seq2seq_results_df = pd.read_csv('../../../runs/wdsq_tunned/google_t5-large-ssm/evaluation/version_4/results.csv')
seq2seq_results_df.head()

answers_cols = seq2seq_results_df.columns[2:-1]
answers_cols

with open('../simplequestions_candidate_selection_dataset/test.pkl', 'rb') as fh:
    test_df = pickle.load(fh)

seq2seq_results_df['question'] = seq2seq_results_df['question'].apply(lambda s: s.replace('\n', ''))

final_test_df = seq2seq_results_df.merge(test_df, left_on='question', right_on='Q')
final_test_df.head()

,question,target,answer_0,answer_1,answer_2,answer_3,answer_4,answer_5,answer_6,answer_7,...,S,P,O,Q,text_with_labeling,ner_entities,mgenre_predicted_entities_list,linked_entities_list,linked_entities_ids_list,one_hop_neighbors
0,Where did roger marquis die,Holyoke,New York City,New York City,Los Angeles,New York City,Los Angeles,Los Angeles,New York City,Los Angeles,...,Q7358590,P20,Q1637790,Where did roger marquis die,Where Did [START] Roger Marquis [END] Die,[roger marquis],"[Roger Marquis >> en, List of Star Trek charac...",[Roger Marquis >> en],[Q7358590],"{'Q7358590': [('Q5', 'Human'), ('Q30', 'U.S. o..."
1,what was the cause of death of yves klein,myocardial infarction,myocardial infarction,myocardial infarction,myocardial infarction,myocardial infarction,myocardial infarction,tuberculosis,pancreatic cancer,lung cancer,...,Q154335,P509,Q12152,what was the cause of death of yves klein,What Was The Cause Of Death Of [START] Yves Kl...,[yves klein],"[Yves Klein >> en, Yves Klein >> fr, Yves Klei...",[Yves Klein >> en],[Q154335],"{'Q154335': [('Q150', 'Franska'), ('Q5', 'Huma..."
2,What position does carlos gomez play?,center fielder,midfielder,defender,defender,midfielder,defender,defender,defender,midfielder,...,Q2747238,P413,Q5059480,What position does carlos gomez play?,What Position Does [START] Carlos Gomez [END] ...,[carlos gomez],"[Carlos Gómez >> en, Carlos Gómez (footballer,...",[Carlos Gomez >> en],[Q2747238],"{'Q2747238': [('Q5', 'Human'), ('Q786', 'Quisq..."
3,how does engelbert zaschka identify,male,Germans,Germans,Germanists,Germans,homosexuality,Jewishness,Germans,Germanists,...,Q62498,P21,Q6581097,how does engelbert zaschka identify,How Does [START] Engelbert Zaschka [END] Identify,[engelbert zaschka],"[Engelbert Zaschka >> en, Engelbert Zaschka >>...",[Engelbert Zaschka >> en],[Q62498],"{'Q62498': [('Q183', 'GERMANY'), ('Q5', 'Human..."
4,what position does pee wee reese play in baseball,shortstop,catcher,catcher,catcher,outfielder,catcher,catcher,catcher,outfielder,...,Q182485,P413,Q1143358,what position does pee wee reese play in baseball,What Position Does [START] Pee Wee Reese [END]...,[pee wee reese],"[Pee Wee Reese >> en, Pee Wee Reese >> de, Pee...",[Pee Wee Reese >> en],[Q182485],"{'Q182485': [('Q5', 'Human'), ('Q30', 'U.S. of..."


In [10]:
for _, row in tqdm(final_test_df.iterrows(), total=final_test_df.index.size):
    start_time = time.time()
    question_entities = [Entity(idx) for idx in row['one_hop_neighbors'].keys()]
    
    answers_candidates = []
    for label in row[answers_cols].unique():
        try:
            answers_candidates.append(Entity.from_label(label)[0])
        except ValueError:
            pass
    
    qtr = QuestionToRank(
        row['question'],
        question_entities,
        answers_candidates,
    )
    qtr.final_answers()

    logger.info({
        'msg': f"Question {row['question']} processed",
        'elapsed_time[ms]': time.time() - start_time,
    })

  0%|          | 0/5186 [00:00<?, ?it/s]
